In [1]:
!pip install python-terrier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.2/335.2 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.8 MB/s eta 0:00:00
  Preparing metadata (setup

In [2]:
import pyterrier as pt
if not pt.started():
  pt.init()

terrier-assemblies 5.8 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8



In [3]:
!pip install Arabic-Stopwords

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 16.3 MB/s eta 0:00:00


In [4]:
import pandas as pd
#to display the full text on the notebook without truncation
pd.set_option('display.max_colwidth', 150)
import numpy as np
import re
from snowballstemmer import stemmer
from tqdm import tqdm
import arabicstopwords.arabicstopwords as stp

In [5]:
#removing Stop Words function
def remove_stop_words(sentence):
    terms=[]
    stopWords= set(stp.stopwords_list())
    for term in sentence.split() :
        if term not in stopWords :
           terms.append(term)
    return " ".join(terms)

#a function to normalize the tweets
def normalize(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    return(text)

#define the stemming function
ar_stemmer = stemmer("arabic")
def stem(sentence):
    return " ".join([ar_stemmer.stemWord(i) for i in sentence.split()])


# perform all preprocessing steps
def preprocess(sentence):
  # apply preprocessing steps on the given sentence
  sentence =remove_stop_words(sentence)
  sentence =normalize(sentence)
  sentence =stem(sentence)
  return sentence


In [6]:
%rm -rf IR-in-Arabic
%rm -rf evetarIndex
!git clone https://github.com/telsayed/IR-in-Arabic.git
!unzip IR-in-Arabic/Summer2021/data/EveTAR/evetarIndex.zip -d evetarIndex
!ls evetarIndex

Cloning into 'IR-in-Arabic'...
remote: Enumerating objects: 710, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 710 (delta 12), reused 0 (delta 0), pack-reused 661
Receiving objects: 100% (710/710), 43.94 MiB | 15.90 MiB/s, done.
Resolving deltas: 100% (189/189), done.
Archive:  IR-in-Arabic/Summer2021/data/EveTAR/evetarIndex.zip
  inflating: evetarIndex/data.direct.bf  
  inflating: evetarIndex/data.document.fsarrayfile  
  inflating: evetarIndex/data.inverted.bf  
  inflating: evetarIndex/data.lexicon.fsomapfile  
  inflating: evetarIndex/data.lexicon.fsomaphash  
  inflating: evetarIndex/data.lexicon.fsomapid  
  inflating: evetarIndex/data.meta.idx  
  inflating: evetarIndex/data.meta.zdata  
  inflating: evetarIndex/data.properties  
data.direct.bf		   data.lexicon.fsomapfile  data.meta.idx
data.document.fsarrayfile  data.lexicon.fsomaphash  data.meta.zdata
data.inverted.bf	   data.lexicon.fsomapid    data.propert

In [7]:
#we will load the index
index_ref = pt.IndexRef.of("./evetarIndex/data.properties")
index = pt.IndexFactory.of(index_ref)

In [8]:
dataset_links=["https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-01.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-02.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-03.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-04.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-05.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-06.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-07.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-08.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-09.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-10.txt"]

full_data=pd.DataFrame()
for i in tqdm(range(len(dataset_links))):
    tweets=pd.read_csv(dataset_links[i], sep='\t')
    full_data=pd.concat([full_data,tweets],ignore_index=True)
full_data.reset_index(inplace=True,drop=True)
#the docno will be our tweetID
full_data["docno"]=full_data["tweetID"].astype(str)
full_data

100%|██████████| 10/10 [00:03<00:00,  2.87it/s]


,tweetID,tweetText,docno
0,549679192804061184,"الاعدام لعامل مطعم قتل زميله طعناً في ""البيادر"" أيدت محكمة التمييز الحكم الصادر عن محكمة الجنايات الكبرى والقاضي... http://t.co/H0txdjv3Kn",549679192804061184
1,549699343666532352,#الأخبار ▪ تأجيل محاكمة 7 إرهابيين بسبب غياب الدفاع: أجلت محكمة الجنايات بالعاصمة إلى تاريخ لاحق محاكمة سبعة إ... http://t.co/GM4jmpAWbR,549699343666532352
2,549711593487888387,@helale9999 عشآن أعطيتك وحده صميم صرت ترمي أعذار ...حقق العالميةة و أرجع كلمني يَ الأياب الانتحاري,549711593487888387
3,549719610459967488,#النهدي ثمانية قتلى في تفجير انتحاري بسيارة مفخخة أمام معملين للغاز في ريف حمص - شبكة الصين http://t.co/r5zFEuzAPu,549719610459967488
4,549720880717508608,البحرين: ضبط مطلوبين متورطين في التفجير بالعكر الشرقي بقية الموضوع اضغط هنا http://t.co/t4A5bNrqyh,549720880717508608
...,...,...,...
49995,561985373048299520,مواسيا الشعب السعودي..حاكم دبي يبدأ جلسة مجلس الوزراء بقراءة الفاتحة على الملك عبدالله #الخبر #السعودية #saudi #ksa,561985373048299520
49996,561987332878766081,@al_shalal @F_D_A82 تم تفجير صماخنا,561987332878766081
49997,561988825186971650,@aubyazid123 جزاك الله ألف خير ❌ جزاك الله خير ✔️ - كلمة ألف فيها تحجير لخير الله.,561988825186971650
49998,561991173360091136,كيف نفّذت «النصرة» عمليّة تفجير الحافلة اللبنانية في دمشق؟ http://t.co/TEmP1Dso1v,561991173360091136


In [9]:
#read the queries file from Github
topics=pd.read_csv("https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/topics.txt", sep='\t',names=['data'])
queries=[]
qid=[]
#we will get the queries and their ids from the topics file
for i in range(len(topics)):
    splitted=topics["data"][i:i+1][i].split(" ")
    if splitted[0]=="<title>":
       queries.append(' '.join(splitted[1:]))
    if splitted[0]=="<num>":
       qid.append(splitted[2])
queriesDF=pd.DataFrame()
#the queries datframe should have qid and query columns to retrieve using PyTerrier
queriesDF["qid"]=qid
queriesDF["raw_query"]=queries
#remove the stopwords from queries, do normalization, and apply stemming
queriesDF["query"]=queriesDF["raw_query"].apply(preprocess)
queriesDF

,qid,raw_query,query
0,E01,تفجير انتحاري في اب,تفجير انتحار اب
1,E02,ليتوانيا تستخدم اليورو بدل الليتاس,ليتواني استخدم يورو بدل ليتاس
2,E03,فلسطين تطلب الانضمام للمحكمة الجنائية الدولية,لسط تطلب انضمام محكمه جناءيه دوليه
3,E04,وفاة أبو أنس الليبي في نيويورك,وفا انس ليب نيويور
4,E05,اختراق كوريا الشمالية حسابات سوني,اختراق كوري شماليه حسابا سون
5,E06,بناء أول كنيسة في إسطنبول منذ قرن,ناء كنيس اسطنبول قرن
6,E07,باباندريو يؤسس حزب جديد,باباندر يءسس حزب جديد
7,E08,بوكو حرام تخطف شباب في نيجريا,وكو حرام تخطف شباب نيجري
8,E09,سيطرة بوكو حرام على قاعدة عسكرية في نيجيريا,سيطر وكو حرام قاعد عسكر نيجيري
9,E10,مقتل مسلحين في غارات في باكستان,مقتل مسلح غارا باكست


***Retrieval with a language model using Jelinek-Mercer smoothing***

In [12]:
JM_retr = pt.BatchRetrieve(index,wmodel="Hiemstra_LM",num_results=10)


In [13]:
query="ليتوانيا تستخدم اليورو بدل الليتاس"
#we need to process the query also as we did for documents
query = preprocess(query)
#we will call the search function using our retrieval model we set up above
results=JM_retr.search(query)
if len(results)==0:
   print("There are no relevant documents for your selected query.")
else:
   print(results)

  qid  docid               docno  rank      score  \
0   1   1438  550604910853509121     0  21.783563   
1   1   1447  550604976439824385     1  21.783563   
2   1   1453  550605488467881984     2  21.783563   
3   1   1454  550605489260593153     3  21.783563   
4   1   1472  550611742779133953     4  21.783563   
5   1   1474  550612983659765760     5  21.783563   
6   1   1646  550671426580000769     6  21.783563   
7   1   2300  550993260253294592     7  21.783563   
8   1   1432  550604875898179585     8  21.003648   
9   1   1475  550613016228556800     9  21.003648   

                           query  
0  ليتواني استخدم يورو بدل ليتاس  
1  ليتواني استخدم يورو بدل ليتاس  
2  ليتواني استخدم يورو بدل ليتاس  
3  ليتواني استخدم يورو بدل ليتاس  
4  ليتواني استخدم يورو بدل ليتاس  
5  ليتواني استخدم يورو بدل ليتاس  
6  ليتواني استخدم يورو بدل ليتاس  
7  ليتواني استخدم يورو بدل ليتاس  
8  ليتواني استخدم يورو بدل ليتاس  
9  ليتواني استخدم يورو بدل ليتاس  


In [14]:
full_data[full_data['docno'].isin(results['docno'].loc[0:4].tolist())]

,tweetID,tweetText,docno
1438,550604910853509121,ليتوانيا تتخلى عن الليتاس وتنضم لليورو http://t.co/qEtr8xlvkE,550604910853509121
1447,550604976439824385,ليتوانيا تتخلى عن الليتاس وتنضم لليورو http://t.co/1oawz8DGs7,550604976439824385
1453,550605488467881984,#news ليتوانيا تتخلى عن الليتاس وتنضم لليورو http://t.co/w2UNHO7gD8 #usa #uk #gcc,550605488467881984
1454,550605489260593153,ليتوانيا تتخلى عن الليتاس وتنضم لليورو http://t.co/EI9JGavlob,550605489260593153
1472,550611742779133953,BBC Arabic - ليتوانيا تتخلى عن الليتاس وتنضم لليورو http://t.co/rilCUEAmxQ,550611742779133953


**Let's update Lambda and set it to 0.95****

In [15]:
JM_retr_highLambda = pt.BatchRetrieve(index,wmodel="Hiemstra_LM",controls ={"c":0.95},num_results=10)

In [17]:
query="ليتوانيا تستخدم اليورو بدل الليتاس"
#we need to process the query also as we did for documents
query = preprocess(query)
#we will call the search function using our retrieval model we set up above
results=JM_retr_highLambda.search(query)
if len(results)==0:
   print("There are no relevant documents for your selected query.")
else:
   print(results)

  qid  docid               docno  rank      score  \
0   1   1438  550604910853509121     0  21.783563   
1   1   1447  550604976439824385     1  21.783563   
2   1   1453  550605488467881984     2  21.783563   
3   1   1454  550605489260593153     3  21.783563   
4   1   1472  550611742779133953     4  21.783563   
5   1   1474  550612983659765760     5  21.783563   
6   1   1646  550671426580000769     6  21.783563   
7   1   2300  550993260253294592     7  21.783563   
8   1   1432  550604875898179585     8  21.003648   
9   1   1475  550613016228556800     9  21.003648   

                           query  
0  ليتواني استخدم يورو بدل ليتاس  
1  ليتواني استخدم يورو بدل ليتاس  
2  ليتواني استخدم يورو بدل ليتاس  
3  ليتواني استخدم يورو بدل ليتاس  
4  ليتواني استخدم يورو بدل ليتاس  
5  ليتواني استخدم يورو بدل ليتاس  
6  ليتواني استخدم يورو بدل ليتاس  
7  ليتواني استخدم يورو بدل ليتاس  
8  ليتواني استخدم يورو بدل ليتاس  
9  ليتواني استخدم يورو بدل ليتاس  


In [18]:
full_data[full_data['docno'].isin(results['docno'].loc[0:4].tolist())]

,tweetID,tweetText,docno
1438,550604910853509121,ليتوانيا تتخلى عن الليتاس وتنضم لليورو http://t.co/qEtr8xlvkE,550604910853509121
1447,550604976439824385,ليتوانيا تتخلى عن الليتاس وتنضم لليورو http://t.co/1oawz8DGs7,550604976439824385
1453,550605488467881984,#news ليتوانيا تتخلى عن الليتاس وتنضم لليورو http://t.co/w2UNHO7gD8 #usa #uk #gcc,550605488467881984
1454,550605489260593153,ليتوانيا تتخلى عن الليتاس وتنضم لليورو http://t.co/EI9JGavlob,550605489260593153
1472,550611742779133953,BBC Arabic - ليتوانيا تتخلى عن الليتاس وتنضم لليورو http://t.co/rilCUEAmxQ,550611742779133953


***We can retrieve the relevant documents to a set of queries***

In [19]:
JM_res=JM_retr.transform(queriesDF)
JM_res

,qid,docid,docno,rank,score,raw_query,query
0,E01,1602,550652866793578496,0,10.747373,تفجير انتحاري في اب,تفجير انتحار اب
1,E01,329,550239745607880704,1,10.053698,تفجير انتحاري في اب,تفجير انتحار اب
2,E01,453,550274033946988544,2,10.053698,تفجير انتحاري في اب,تفجير انتحار اب
3,E01,1376,550582854749790208,3,10.053698,تفجير انتحاري في اب,تفجير انتحار اب
4,E01,1466,550609859712794625,4,10.053698,تفجير انتحاري في اب,تفجير انتحار اب
...,...,...,...,...,...,...,...
495,E65,45458,560895659381780480,5,19.185313,إعادة تشكيل مجلس الوزراء السعودي,اعاد تشكيل مجلس وزراء سعود
496,E65,46007,560958826325622784,6,19.185313,إعادة تشكيل مجلس الوزراء السعودي,اعاد تشكيل مجلس وزراء سعود
497,E65,45429,560893357514559488,7,18.677957,إعادة تشكيل مجلس الوزراء السعودي,اعاد تشكيل مجلس وزراء سعود
498,E65,45142,560878203040837632,8,18.566039,إعادة تشكيل مجلس الوزراء السعودي,اعاد تشكيل مجلس وزراء سعود


In [20]:
JM_highLambda_res=JM_retr_highLambda.transform(queriesDF)
JM_highLambda_res

,qid,docid,docno,rank,score,raw_query,query
0,E01,1602,550652866793578496,0,10.747373,تفجير انتحاري في اب,تفجير انتحار اب
1,E01,329,550239745607880704,1,10.053698,تفجير انتحاري في اب,تفجير انتحار اب
2,E01,453,550274033946988544,2,10.053698,تفجير انتحاري في اب,تفجير انتحار اب
3,E01,1376,550582854749790208,3,10.053698,تفجير انتحاري في اب,تفجير انتحار اب
4,E01,1466,550609859712794625,4,10.053698,تفجير انتحاري في اب,تفجير انتحار اب
...,...,...,...,...,...,...,...
495,E65,45458,560895659381780480,5,19.185313,إعادة تشكيل مجلس الوزراء السعودي,اعاد تشكيل مجلس وزراء سعود
496,E65,46007,560958826325622784,6,19.185313,إعادة تشكيل مجلس الوزراء السعودي,اعاد تشكيل مجلس وزراء سعود
497,E65,45429,560893357514559488,7,18.677957,إعادة تشكيل مجلس الوزراء السعودي,اعاد تشكيل مجلس وزراء سعود
498,E65,45142,560878203040837632,8,18.566039,إعادة تشكيل مجلس الوزراء السعودي,اعاد تشكيل مجلس وزراء سعود


### **Evaluating our results**

In [21]:
qrels=pd.read_csv("https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/qrels.txt", sep='\t',names=['qid','Q0','docno','label'])
qrels['docno']=qrels['docno'].astype(str)
qrels = qrels[qrels["docno"].isin(full_data["docno"].tolist())]
qrels

,qid,Q0,docno,label
0,E01,Q0,549711593487888387,0
1,E01,Q0,549719610459967488,0
2,E01,Q0,549720880717508608,0
4,E01,Q0,549780004230283264,0
7,E01,Q0,549880138033135617,0
...,...,...,...,...
61940,E65,Q0,561984084948176896,0
61941,E65,Q0,561984303219744768,0
61942,E65,Q0,561984392516472833,0
61944,E65,Q0,561984778438590464,0


To evaluate our results we can use Pyterrier Utils.evaluate function. This function take the results and the qrels dataframe containing three columns which are qid, docno, label.



In [22]:
eval = pt.Utils.evaluate(JM_res,qrels[['qid','docno','label']],metrics=["map","recall","P"])
eval

<ipython-input-22-eb0b73b6a88e>:1: DeprecationWarning: Call to deprecated function (or staticmethod) evaluate. (Use pt.Evaluate instead) -- Deprecated since version 0.9.
  eval = pt.Utils.evaluate(JM_res,qrels[['qid','docno','label']],metrics=["map","recall","P"])


{'map': 0.03312589002472813,
 'R@5': 0.01822545865120636,
 'R@10': 0.03640268549225227,
 'R@15': 0.03640268549225227,
 'R@20': 0.03640268549225227,
 'R@30': 0.03640268549225227,
 'R@100': 0.03640268549225227,
 'R@200': 0.03640268549225227,
 'R@500': 0.03640268549225227,
 'R@1000': 0.03640268549225227,
 'P@5': 0.8640000000000001,
 'P@10': 0.8579999999999999,
 'P@15': 0.5720000000000004,
 'P@20': 0.42899999999999994,
 'P@30': 0.2860000000000002,
 'P@100': 0.0858,
 'P@200': 0.0429,
 'P@500': 0.01716000000000001,
 'P@1000': 0.008580000000000004}

 compare between different retrieval models using PyTerrier

In [25]:
JM_retr = pt.BatchRetrieve(index,wmodel="Hiemstra_LM",controls ={"c":0.15},num_results=1000)
JM_retr_highLambda = pt.BatchRetrieve(index,wmodel="Hiemstra_LM",controls ={"c":0.95},num_results=1000)
bm25_retr = pt.BatchRetrieve(index, controls = {"wmodel": "BM25"},num_results=1000)
tfidf_retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"},num_results=1000)
#call pt.Experiment
pt.Experiment(
[JM_retr ,JM_retr_highLambda, bm25_retr, tfidf_retr],
queriesDF,
qrels,
eval_metrics=["map", "P"],
names=["JM_retr","JM_retr_highLambda","bm25_retr","tfidf_retr"]
)


,name,map,P@5,P@10,P@15,P@20,P@30,P@100,P@200,P@500,P@1000
0,JM_retr,0.593203,0.864,0.860,0.857333,0.841,0.826000,0.7502,0.6561,0.48064,0.30564
1,JM_retr_highLambda,0.593203,0.864,0.860,0.857333,0.841,0.826000,0.7502,0.6561,0.48064,0.30564
2,bm25_retr,0.596614,0.864,0.858,0.854667,0.853,0.842667,0.7498,0.6600,0.48252,0.30512
3,tfidf_retr,0.595304,0.864,0.858,0.850667,0.852,0.842667,0.7490,0.6597,0.48128,0.30432


In [26]:
#perquery(bool): If True return each metric for each query, else return mean metricsacross all queries. Default=False
pt.Experiment(
[JM_retr,JM_retr_highLambda ,bm25_retr, tfidf_retr],
queriesDF,
qrels,
eval_metrics=["map", "P"],
names=["JM_retr","JM_retr_highLambda","bm25_retr","tfidf_retr"],
perquery=True
)

,name,qid,measure,value
0,JM_retr,E01,map,0.656853
1,JM_retr,E01,P@5,1.000000
2,JM_retr,E01,P@10,0.900000
3,JM_retr,E01,P@15,0.866667
4,JM_retr,E01,P@20,0.750000
...,...,...,...,...
1995,tfidf_retr,E65,P@30,1.000000
1996,tfidf_retr,E65,P@100,0.970000
1997,tfidf_retr,E65,P@200,0.870000
1998,tfidf_retr,E65,P@500,0.532000


In [27]:
#dataframe(bool): If True return results as a dataframe. Else as a dictionary of dictionaries. Default=True.
pt.Experiment(
[JM_retr,JM_retr_highLambda ,bm25_retr, tfidf_retr],
queriesDF,
qrels,
eval_metrics=["map", "P"],
names=["JM_retr","JM_retr_highLambda","bm25_retr","tfidf_retr"],
dataframe=False
)

{'JM_retr': [0.5932031234026583,
  0.8640000000000001,
  0.8599999999999999,
  0.8573333333333332,
  0.8409999999999997,
  0.8259999999999998,
  0.7502,
  0.6561,
  0.48064000000000007,
  0.30564],
 'JM_retr_highLambda': [0.5932031234026583,
  0.8640000000000001,
  0.8599999999999999,
  0.8573333333333332,
  0.8409999999999997,
  0.8259999999999998,
  0.7502,
  0.6561,
  0.48064000000000007,
  0.30564],
 'bm25_retr': [0.5966141627270125,
  0.8640000000000002,
  0.8579999999999997,
  0.8546666666666666,
  0.853,
  0.8426666666666663,
  0.7497999999999999,
  0.66,
  0.48252,
  0.30511999999999995],
 'tfidf_retr': [0.5953039664007528,
  0.8640000000000002,
  0.8579999999999997,
  0.8506666666666665,
  0.8519999999999999,
  0.8426666666666663,
  0.7490000000000001,
  0.6597,
  0.4812800000000001,
  0.30432]}